In [3]:
import numpy as np

In [31]:
class DataNode():
    def __init__(self, activity_patterns):
        self.n_neurons=activity_patterns.shape[0]
        self.activity_patterns=activity_patterns
        
class SimuNode():
    def __init__(self,n_neurons):
        self.n_neurons=n_neurons
        
class Synapse():
    def __init__(self,pre_synaptic_node, post_synaptic_node,syn_type='exc',plasticity=None):
        self.pre_synaptic_node=pre_synaptic_node
        self.post_synaptic_node=post_synaptic_node
        self.syn_type=syn_type
        if plasticity=='BCM':
            self.theta_BCM = 0
            self.theta_BCM_dt = .001
            self.BCM_target = 5000.0
            self.alpha = 0.01
        
    def connect(self):
        row_dim=self.pre_synaptic_node.n_neurons
        col_dim=self.post_synaptic_node.n_neurons
        k=7.5
        theta=1.0
        if self.syn_type=='inh':
            self.syn_weights=-np.random.gamma(shape=k,scale=theta,size=(col_dim,row_dim))
            #self.syn_weights=-np.random.normal(loc=0,size=(col_dim,row_dim))
        elif self.syn_type=='exc':
            self.syn_weights=np.random.gamma(shape=k,scale=theta,size=(col_dim,row_dim))
            
    def learn(self,pre_syn_activities,post_syn_activities):
        if plasticity=='BCM':
            self._update_weights_BCM(pre_syn_activities,post_syn_activities)
            self._update_BCM_threshold(post_syn_activities)
        
    def _update_weights_BCM(self,pre_syn_activities,post_syn_activities):
        print('W before',self.syn_weights.shape)
        #print('syn',pre_syn_activity_pattern.reshape(9479,1)@post_syn_activity_pattern.reshape(1,2))
        self.syn_weights+= self.alpha*pre_syn_activities@post_syn_activities@(post_syn_activity_pattern-self.theta_BCM)
        print('W after',self.syn_weights.shape)
        
    def _update_BCM_threshold(self,activity_pattern):
        self.theta_BCM += self.theta_BCM_dt*((activity_pattern/self.BCM_target)*activity_pattern - self.theta_BCM)
        
class Simulate():
    def __init__(self,init_dict,data):
        self.init_dict=init_dict
        self.connectivity_graph=self.init_dict['connectivity_graph']
        self.node_classification=self.init_dict['node_classification']
        self.node_ns=self.init_dict['node_ns']
        self.data=data
        self.data_dict={}
        self.sim_dict={}
        dat_ind=0
        #exc_dat_ind=0
        sim_ind=0
        #exc_sim_ind=0
        self.prev_timepoint=[]
        for j in range(len(self.node_classification)):
            if self.node_classification[j]=='data_inh':
                self.data_dict[j]=dat_ind
                dat_ind+=1
            if self.node_classification[j]=='data_exc':
                self.data_dict[j]=dat_ind
                dat_ind+=1
            if self.node_classification[j]=='simulated_exc':
                self.sim_dict[j]=sim_ind
                self.prev_timepoint.append(np.zeros(self.node_ns[j],))
                sim_ind+=1
            if self.node_classification[j]=='simulated_inh':
                self.sim_dict[j]=sim_ind
                self.prev_timepoint.append(np.zeros(self.node_ns[j],))
                sim_ind+=1
        
    def _connect(self):
        node_lst=[]
        for node_ind,node in enumerate(self.node_classification):
            print(node)
            if node=='simulated_inh':
                node_lst.append(SimuNode(self.node_ns[node_ind]))
            if node=='simulated_exc':
                node_lst.append(SimuNode(self.node_ns[node_ind]))
            if node=='data_inh':
                ind=self.data_dict[node_ind]
                node_lst.append(DataNode(self.data[ind]))
            if node=='data_exc':
                ind=self.data_dict[node_ind]
                node_lst.append(DataNode(self.data[ind]))
        self.syn_dict_pre={}
        self.syn_dict_post={}
        i=0
        self.syn_lst=[]
        for pre in range(0,self.connectivity_graph.shape[0]):
            for post in range(0,self.connectivity_graph.shape[1]):
                if self.connectivity_graph[pre,post]==1:
                    if node_classification[pre][-3:]=='exc':
                        syn_type='exc'
                    elif node_classification[pre][-3:]=='inh':
                        syn_type='inh'
                    #Synapses cannot exist between data nodes, ensure that!
                    syn=Synapse(node_lst[pre],node_lst[post],syn_type)
                    syn.connect()
                    self.syn_dict_pre[i]=pre
                    self.syn_dict_post[i]=post
                    self.syn_lst.append(syn)
                    i+=1
        
    def one_step(self,t):
        self._connect()
        sim_one_t=[]
        nodes_to_update=[]
        nodes_dims=[]
        for i,node in enumerate(self.node_classification):
            if node[:3]=='sim':
                nodes_to_update.append(i)
                nodes_dims.append(self.node_ns[i])
        for i,node in enumerate(nodes_to_update):
            inds=self.connectivity_graph[:,i].flatten()
            weights_stack=np.zeros((nodes_dims[i],1))
            act_stack=np.zeros((1,))
            for syn_ind,syn in enumerate(self.syn_lst):
                if self.syn_dict_post[syn_ind]==node:
                    pre_ind=self.syn_dict_pre[syn_ind]
                    weights_stack=np.hstack((weights_stack,syn.syn_weights))
                    act_stack=np.hstack((act_stack,data[pre_ind][:,t].flatten()))
            weights_stack=weights_stack[:,1:]
            act_stack=act_stack[1:]
            activation=-self.prev_timepoint[i]+weights_stack@act_stack
            self.prev_timepoint[i]=activation
            print('activation',activation)

    def simulate(self):
        for t in range(0,10):
            self.one_step(t)
            
        
            
        
        

In [33]:
node_classification=['data_exc','data_inh','simulated_exc']
connectivity_graph=np.array([[0,0,1],[0,0,1],[0,0,0]])
node_ns=[exc_dat.shape[0],inh_dat.shape[0],4]
data=[exc_dat,inh_dat]

init_dict={'node_classification':node_classification,'connectivity_graph':connectivity_graph,'node_ns':node_ns}
run=Simulate(init_dict,data).simulate()


data_exc
data_inh
simulated_exc
activation [283094.27179213 264534.46562955 285748.04894221 267665.66199399]
data_exc
data_inh
simulated_exc
activation [ 6153.08776438 21430.67898981 -4034.10315616 30761.92483544]
data_exc
data_inh
simulated_exc
activation [252487.79243391 235105.66151673 258020.25465777 200558.14723478]
data_exc
data_inh
simulated_exc
activation [-148169.47179039 -130829.59649556 -146274.46930121  -92746.39285298]
data_exc
data_inh
simulated_exc
activation [279649.26707198 245478.24720172 275948.66214463 217294.67571441]
data_exc
data_inh
simulated_exc
activation [-34425.14856744   2122.19094801  -6299.35568191  48849.78200224]
data_exc
data_inh
simulated_exc
activation [333152.31431745 281048.9770522  292193.97347704 260219.1392964 ]
data_exc
data_inh
simulated_exc
activation [-209869.37806377 -157363.56782638 -163903.13452516 -131400.36646018]
data_exc
data_inh
simulated_exc
activation [346076.33049198 289461.69251646 293821.44647705 261128.04779584]
data_exc
data_i

In [6]:
data_path='/media/maria/DATA1/Documents/data_for_suite2p/TX39_total/'
dt=1
spks= np.load(data_path+'spks.npy')
print('Shape of the data matrix, neurons by timepoints:',spks.shape)
iframe = np.load(data_path+'iframe.npy') # iframe[n] is the microscope frame for the image frame n
ivalid = iframe+dt<spks.shape[-1] # remove timepoints outside the valid time range
iframe = iframe[ivalid]
S = spks[:, iframe+dt]
#U=np.load('U.npy')

Shape of the data matrix, neurons by timepoints: (18795, 30766)


In [7]:
U=np.load('U.npy')

In [9]:
#del spks
del S

In [8]:
stim_ens_inds=np.nonzero(U[:,13])[0]
exc_dat=S[stim_ens_inds]
inh_dat=S[np.random.randint(0,S.shape[0],size=100),:]

In [12]:
print(exc_dat.shape)

(642, 30560)


In [9]:
inh_dat=S[np.random.randint(0,S.shape[0],size=10),:]
exc_dat=S[np.random.randint(0,S.shape[0],size=100),:]

In [7]:
np.random.randint(0,100,size=100)

array([63, 86, 49,  1, 55, 81,  2, 97, 27, 53, 32, 11, 35, 41, 51, 93, 31,
       52, 28, 89, 70, 17, 46, 85, 37, 49, 70, 78, 57, 91, 47, 70, 50, 60,
       87, 81, 76, 39,  3, 62, 64, 70,  2, 72, 73, 72, 71, 89,  5, 73, 53,
       47,  8,  6, 33, 33, 23, 79, 30, 72, 60, 43, 84, 58, 20, 13, 93, 48,
       71, 90, 29, 88, 77, 26, 48, 82, 65, 51, 60, 22,  0, 96,  1, 53, 21,
       58, 34, 58, 59, 24, 31, 11, 20, 25, 43, 12, 11, 76, 93, 38])